In [1]:
import pandas as pd

make a variables for the list of candidate mvp winner from 1991 - 2024

In [2]:
mvps = pd.read_csv('new_mvps_ws.csv')

In [3]:
mvps = mvps[["Player",'year', 'Pts Won', 'Pts Max', 'Share']]

make a variables for player game per season (normal stats)

In [4]:
players = pd.read_csv('player_1991_2024.csv')

In [5]:
players.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'year'],
      dtype='object')

In [6]:
del players['Rk']

this * on player means he is the candidate of mvp winner, since i dont want that, ill replace it

In [7]:
players['Player'] = players['Player'].str.replace('*', "", regex=False)

make a variables for player game per season (advanced stats)

In [8]:
advanced_players = pd.read_csv('player_stats_1991_2024.csv')

delete unnecessary col

In [9]:
del advanced_players['Unnamed: 19']
del advanced_players['Unnamed: 24']
del advanced_players['Rk']

if u look closesly some of players have more than 1 team, that mean in a season he changed teams, so in order to make processing the data easier,
ill take the last team he played for that season

In [10]:
def single_row(df):
    ''' 
    check if there are more than 1 name and stat in data frame
    if there is its gonna return the last team he played
    '''
    if df.shape[0] == 1:
        return df
    else:
        row = df[df['Tm'] == 'TOT']
        row['Tm'] = df.iloc[-1,:]['Tm']
        return row

grouping by player and row

In [11]:
players = players.groupby(['Player', 'year']).apply(single_row)

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_33120\2294915571.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  players = players.groupby(['Player', 'year']).apply(single_row)


since the data groupby on player and year , we need to drop the index, so it can rever back to the original index

In [12]:
players.index = players.index.droplevel()
players.index = players.index.droplevel()

this * on player means he is the candidate of mvp winner, since i dont want that, ill replace it

In [13]:
advanced_players['Player'] = advanced_players['Player'].str.replace('*', "", regex=False)

In [14]:
advanced_players = advanced_players.groupby(['Player', 'year']).apply(single_row)

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_33120\3590247182.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  advanced_players = advanced_players.groupby(['Player', 'year']).apply(single_row)


In [15]:
advanced_players.index = advanced_players.index.droplevel()
advanced_players.index = advanced_players.index.droplevel()

to minimize the redundant of the column, since im gonna concat the normal stats and advanced stat, i need to drop some of the col

In [16]:
advanced_players = advanced_players.drop(['Pos', 'Age', 'Tm', 'G', 'MP'],axis=1)

In [17]:
advanced_players_to_fill = ['PER', 'TS%', '3PAr', 'FTr', 'ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%','WS/48']

so some of the data was nan, its because their attempted shot is 0, thats why its nan, i check every thing, all clear that the reason why it was nan
all of em by the attempt was 0

In [18]:
advanced_players[advanced_players_to_fill]= advanced_players[advanced_players_to_fill].fillna(0)

In [19]:
players_to_fill = ['FG%','3P%','2P%','eFG%','FT%']

In [23]:
players[players['3P%'].isnull()][['Player','3P%','3PA']]

,Player,3P%,3PA
4510,A.J. Bramlett,NaN,0.0
10094,Aaron Gray,NaN,0.0
10724,Aaron Gray,NaN,0.0
11300,Aaron Gray,NaN,0.0
11844,Aaron Gray,NaN,0.0
...,...,...,...
5866,Željko Rebrača,NaN,0.0
6347,Željko Rebrača,NaN,0.0
6907,Željko Rebrača,NaN,0.0
7485,Željko Rebrača,NaN,0.0


In [20]:
players[players_to_fill] = players[players_to_fill].fillna(0)

In [21]:
merge_players = players.merge(advanced_players,how='inner',on=['Player', 'year'])

In [22]:
merge_players.shape

(15808, 50)

In [23]:
mvps[mvps['year']==2024]

,Player,year,Pts Won,Pts Max,Share
499,Nikola Jokić,2024,926,990,0.935
500,Shai Gilgeous-Alexander,2024,640,990,0.646
501,Luka Dončić,2024,566,990,0.572
502,Giannis Antetokounmpo,2024,192,990,0.194
503,Jalen Brunson,2024,142,990,0.143
504,Jayson Tatum,2024,86,990,0.087
505,Anthony Edwards,2024,18,990,0.018
506,Domantas Sabonis,2024,3,990,0.003
507,Kevin Durant,2024,1,990,0.001


In [24]:
combined = merge_players.merge(mvps, how='outer', on=['Player', 'year'])

In [25]:
combined.shape

(15808, 53)

this coloumn (pts won, pts max, share), its indicate the candidate of MVP, it ranges from 0-1, 0 means they are not selected as mvp candidates,
so there are some np.nan on the value of those coloumn, and i want to fill it as 0

In [26]:
mvps_to_fill = ['Pts Won','Pts Max','Share']

In [27]:
combined[mvps_to_fill] = combined[mvps_to_fill].fillna(0)

In [28]:
combined.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,2.9,6.1,0.135,0.1,-0.2,0.0,1.1,0.0,0.0,0.0
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,0.476,...,2.9,8.8,0.145,1.7,-1.0,0.7,2.0,0.0,0.0,0.0
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,0.537,...,2.6,8.6,0.147,1.6,-0.9,0.7,1.9,0.0,0.0,0.0
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,0.502,...,2.6,9.3,0.157,1.8,-1.2,0.6,1.8,0.0,0.0,0.0
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,2.1,6.7,0.120,0.8,-0.9,-0.1,1.3,0.0,0.0,0.0


make variables for team

In [29]:
teams = pd.read_csv('all_teams.csv')

In [30]:
teams.head()

,Unnamed: 0,W,L,W/L%,GB,PS/G,PA/G,SRS,year,team
0,0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,1991,Atlantic Division
1,1,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
2,2,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
3,3,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
4,4,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets


In [31]:
# if there is string that contain division get rid of that (using negation, so dont have to use if else)
teams = teams[~teams['W'].str.contains('Division')]

In [32]:
teams['team'] = teams['team'].str.replace('*', '', regex = False)

In [33]:
teams.head(30)

,Unnamed: 0,W,L,W/L%,GB,PS/G,PA/G,SRS,year,team
1,1,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics
2,2,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers
3,3,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks
4,4,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
5,5,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
6,6,24,58,.293,32.0,101.8,107.8,-5.91,1991,Miami Heat
8,8,61,21,.744,—,110.0,101.0,8.57,1991,Chicago Bulls
9,9,50,32,.610,11.0,100.1,96.8,3.08,1991,Detroit Pistons
10,10,48,34,.585,13.0,106.4,104.0,2.33,1991,Milwaukee Bucks
11,11,43,39,.524,18.0,109.8,109.0,0.72,1991,Atlanta Hawks


delete unnecessary coloumn

In [34]:
del teams['Unnamed: 0']

In [35]:
with open('nicknames.csv') as f:
    lines = f.readlines()
lines

['Team,Abbreviation\n',
 'Atlanta Hawks,ATL\n',
 'Boston Celtics,BOS\n',
 'Brooklyn Nets,BRK\n',
 'Charlotte Bobcats,CHA\n',
 'Chicago Bulls,CHI\n',
 'Cleveland Cavaliers,CLE\n',
 'Dallas Mavericks,DAL\n',
 'Denver Nuggets,DEN\n',
 'Detroit Pistons,DET\n',
 'Golden State Warriors,GSW\n',
 'Houston Rockets,HOU\n',
 'Indiana Pacers,IND\n',
 'Los Angeles Clippers,LAC\n',
 'Los Angeles Lakers,LAL\n',
 'Memphis Grizzlies,MEM\n',
 'Miami Heat,MIA\n',
 'Milwaukee Bucks,MIL\n',
 'Minnesota Timberwolves,MIN\n',
 'New Orleans Pelicans,NOP\n',
 'New York Knicks,NYK\n',
 'Oklahoma City Thunder,OKC\n',
 'Orlando Magic,ORL\n',
 'Philadelphia 76ers,PHI\n',
 'Portland Trail Blazers,POR\n',
 'Sacramento Kings,SAC\n',
 'San Antonio Spurs,SAS\n',
 'Toronto Raptors,TOR\n',
 'Utah Jazz,UTA\n',
 'Washington Wizards,WAS\n',
 'Phoenix Suns,PHO\n',
 'Washington Bullets,WSB\n',
 'New Orleans Hornets,NOH\n',
 'Charlotte Hornets,CHO\n',
 'Vancouver Grizzlies,VAN\n',
 'Seattle SuperSonics,SEA\n',
 'New Jersey Nets

In [36]:
nicknames = {}

with open('nicknames.csv') as f:
    lines = f.readlines()

    for line in lines[1:]: #skipping the first row coz its the tittle
        Team, Abbreviation  = line.replace('\n','').split(',')
        nicknames[Abbreviation] = Team

i need to map based on the agrevation of team, thats why i need a nickname files, so the map can succesfully operated

In [37]:
combined['team'] = combined['Tm'].map(nicknames)

In [38]:
combined.head(5)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share,team
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,0.476,...,6.1,0.135,0.1,-0.2,0.0,1.1,0.0,0.0,0.0,Los Angeles Lakers
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,0.476,...,8.8,0.145,1.7,-1.0,0.7,2.0,0.0,0.0,0.0,Los Angeles Lakers
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,0.537,...,8.6,0.147,1.6,-0.9,0.7,1.9,0.0,0.0,0.0,Los Angeles Lakers
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,0.502,...,9.3,0.157,1.8,-1.2,0.6,1.8,0.0,0.0,0.0,Phoenix Suns
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,0.504,...,6.7,0.120,0.8,-0.9,-0.1,1.3,0.0,0.0,0.0,Phoenix Suns


In [39]:
combined.shape

(15808, 54)

In [40]:
stats = combined.merge(teams, how='outer',on=(['team', 'year']))

In [41]:
stats.tail(100)

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,Pts Max,Share,team,W,L,W/L%,GB,PS/G,PA/G,SRS
15708,Anžejs Pasečņiks,C,24,WAS,27,0,16.2,2.3,4.3,0.526,...,0.0,0.0,Washington Wizards,25,47,.347,18.5,114.4,119.1,-5.24
15709,Bradley Beal,SG,26,WAS,57,57,36.0,10.4,22.9,0.455,...,0.0,0.0,Washington Wizards,25,47,.347,18.5,114.4,119.1,-5.24
15710,C.J. Miles,SF,32,WAS,10,0,16.1,1.9,5.9,0.322,...,0.0,0.0,Washington Wizards,25,47,.347,18.5,114.4,119.1,-5.24
15711,Dāvis Bertāns,PF,27,WAS,54,4,29.3,4.9,11.3,0.434,...,0.0,0.0,Washington Wizards,25,47,.347,18.5,114.4,119.1,-5.24
15712,Garrison Mathews,SG,23,WAS,18,0,12.6,1.3,3.1,0.429,...,0.0,0.0,Washington Wizards,25,47,.347,18.5,114.4,119.1,-5.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15803,Marvin Bagley III,C,24,WAS,50,25,21.1,4.8,8.2,0.586,...,0.0,0.0,Washington Wizards,15,67,.183,32.0,113.7,123.0,-9.29
15804,Patrick Baldwin Jr.,SF,21,WAS,38,7,13.0,1.6,4.1,0.381,...,0.0,0.0,Washington Wizards,15,67,.183,32.0,113.7,123.0,-9.29
15805,Richaun Holmes,C-PF,30,WAS,40,10,13.9,2.1,3.7,0.558,...,0.0,0.0,Washington Wizards,15,67,.183,32.0,113.7,123.0,-9.29
15806,Tristan Vukcevic,C,20,WAS,10,4,15.3,2.9,6.7,0.433,...,0.0,0.0,Washington Wizards,15,67,.183,32.0,113.7,123.0,-9.29


In [42]:
stats.shape

(15808, 61)

In [43]:
stats.dtypes

Player    object
Pos       object
Age        int64
Tm        object
G          int64
           ...  
W/L%      object
GB        object
PS/G      object
PA/G      object
SRS       object
Length: 61, dtype: object

convert all type to numeeric, make it easier when training the data

In [44]:
stats = stats.apply(pd.to_numeric, errors='ignore')

C:\Users\Wahyunan\AppData\Local\Temp\ipykernel_14560\3935204516.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  stats = stats.apply(pd.to_numeric, errors='ignore')


In [45]:
stats.dtypes

Player     object
Pos        object
Age         int64
Tm         object
G           int64
           ...   
W/L%      float64
GB         object
PS/G      float64
PA/G      float64
SRS       float64
Length: 61, dtype: object

this - on gameback column means, the team has most win than other teams

In [46]:
stats['GB'].unique()

array(['18.0', '29.0', '14.0', '—', '10.0', '26.0', '13.0', '12.0', '2.0',
       '28.0', '27.0', '17.0', '15.0', '33.0', '46.0', '6.0', '22.0',
       '16.0', '20.0', '21.0', '3.0', '11.0', '25.0', '19.0', '5.0',
       '23.0', '12.5', '9.0', '8.0', '4.0', '35.0', '31.0', '1.0', '7.0',
       '32.0', '41.0', '24.0', '39.0', '45.0', '36.0', '37.0', '38.0',
       '4.5', '30.0', '43.0', '25.5', '44.0', '40.0', '42.0', '2.5',
       '51.0', '34.0', '20.5', '32.5', '48.0', '52.0', '3.5', '56.0',
       '10.5', '47.0', '11.5', '22.5', '1.5', '50.0', '21.5', '18.5'],
      dtype=object)

so im gonna convert it to zero

In [47]:
stats['GB'] = stats['GB'].str.replace('—', '0')

this number indicates the disparancy / different between the team that has most winning and the selected team

In [48]:
stats['GB'].unique()

array(['18.0', '29.0', '14.0', '0', '10.0', '26.0', '13.0', '12.0', '2.0',
       '28.0', '27.0', '17.0', '15.0', '33.0', '46.0', '6.0', '22.0',
       '16.0', '20.0', '21.0', '3.0', '11.0', '25.0', '19.0', '5.0',
       '23.0', '12.5', '9.0', '8.0', '4.0', '35.0', '31.0', '1.0', '7.0',
       '32.0', '41.0', '24.0', '39.0', '45.0', '36.0', '37.0', '38.0',
       '4.5', '30.0', '43.0', '25.5', '44.0', '40.0', '42.0', '2.5',
       '51.0', '34.0', '20.5', '32.5', '48.0', '52.0', '3.5', '56.0',
       '10.5', '47.0', '11.5', '22.5', '1.5', '50.0', '21.5', '18.5'],
      dtype=object)

In [49]:
stats['GB'] = pd.to_numeric(stats['GB'])

In [50]:
stats.dtypes

Player     object
Pos        object
Age         int64
Tm         object
G           int64
           ...   
W/L%      float64
GB        float64
PS/G      float64
PA/G      float64
SRS       float64
Length: 61, dtype: object

In [51]:
stats.shape

(15808, 61)

In [52]:
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'year', 'PER',
       'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
       'VORP', 'Pts Won', 'Pts Max', 'Share', 'team', 'W', 'L', 'W/L%', 'GB',
       'PS/G', 'PA/G', 'SRS'],
      dtype='object')

In [53]:
stats.isnull().sum()

Player    0
Pos       0
Age       0
Tm        0
G         0
         ..
W/L%      0
GB        0
PS/G      0
PA/G      0
SRS       0
Length: 61, dtype: int64

save the final clean data to csv

In [54]:
stats.to_csv('final_nba_clean.csv')